In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mybiotools as mbt
import pr_peaks

# 2018-02-20 Bare chair model

Here I want to look at what are the essential features to reproduce the non-monotonic behaviour that I observed in the ratio between H and L sites in the ChIP-seq experiments. Clearly there were too many parameters in the previous edition. Here I want to look at only one site competing with some other sites and looking at whether there are any interesting effects that one can speculate upon.

In [ ]:
# init constants of the system
N = 100                                    # number of equivalent systems
n = 100                                    # number of sites in each system

In [ ]:
# init taus
site_taus = 2.0 * np.ones(n)
Hsite = 49
site_taus[Hsite] = 20.0

In [ ]:
nsteps = 500
sigma = None
phi = np.array([0.01,0.10,0.25,0.50,0.75])
mus = (n*phi).astype(np.int32)
print mus

In [ ]:
%%time
Tflat = np.ones((n,n))/n
flat = pr_peaks.JumpingModel(Tflat,site_taus)
for mu in mus :
    mbt.log_message('Flat','mu = %d'%(mu))
    omega_t_initial = pr_peaks.init_omega_t(N,n,mu)
    flat.run(nsteps,mu,sigma,omega_t_initial)

In [ ]:
def H_to_L(model,Hsites,Lsites) :
    mus = model.occupancy.keys()
    mus.sort()
    nmus = len(mus)
    model.avH = {}
    model.avL = {}
    model.H_to_L = np.zeros(nmus)
    for i,mu in enumerate(mus) :
        model.avH[mu] = model.occupancy[mu][Hsites].mean()
        model.avL[mu] = model.occupancy[mu][Lsites].mean()
        model.H_to_L[i] = model.avH[mu]/model.avL[mu]

In [ ]:
Lsites = [i for i in xrange(n)]
Lsites.remove(Hsite)

In [ ]:
# prepare the transition matrix
THL = np.ones((n,n))
for site in Lsites :
    THL[Hsite,site] += 1
    THL[site,Hsite] += 1
THL = mbt.row_normalize_matrix(THL)

In [ ]:
%%time
HL = pr_peaks.JumpingModel(THL,site_taus)
for mu in mus :
    mbt.log_message('HL','mu = %d'%(mu))
    omega_t_initial = pr_peaks.init_omega_t(N,n,mu)
    HL.run(nsteps,mu,sigma,omega_t_initial)

In [ ]:
H_to_L(HL,[Hsite],Lsites)
H_to_L(flat,[Hsite],Lsites)

In [ ]:
plt.plot(mus,flat.H_to_L,'r')
plt.plot(mus,HL.H_to_L,'b')
plt.xlabel(r'$\mu$')
plt.ylabel('H to L ratio')
plt.axhline(y=1.0,linestyle='--',color='k')
plt.show()

This analysis shows that there is little possibility of explaining the non-monotonic behaviour of the system based upon the enhancement of L->H transitions. There must be another explanation for these phenomena.

### Intermezzo: profiling the code

The execution time of the code above is frustratingly long. Therefore I'd like to do the ancient thing of tryig to figure out what is the reason for this, by profiling the code.

In [ ]:
import cProfile
cProfile.run('HL.run(nsteps,50,sigma,omega_t_initial)')

The result of this analysis is that there is a lot of time spent in the "np.random.choice" function. There would really be little or no benefit in trying to optimize calls to that function, so I leave it here and desist.

## Introducing M sites

Could the non-monotonic behaviour due to the fact that there are intermediate sites, M sites, that get progressively populated as the concentration increases, giving the appearance of an H site enhancement?

In [ ]:
# init taus
site_taus = 2.0 * np.ones(n)
Hsite = 49
Msites = [10,20,30]
site_taus[Hsite] = 20.0
for site in Msites :
    site_taus[site] = 5.0

In [ ]:
%%time
Tflat = np.ones((n,n))/n
flat = pr_peaks.JumpingModel(Tflat,site_taus)
for mu in mus :
    mbt.log_message('Flat','mu = %d'%(mu))
    omega_t_initial = pr_peaks.init_omega_t(N,n,mu)
    flat.run(nsteps,mu,sigma,omega_t_initial)

In [ ]:
Lsites = [i for i in xrange(n)]
Lsites.remove(Hsite)
for site in Msites :
    Lsites.remove(site)

In [ ]:
THL = np.ones((n,n))
for site in Lsites :
    THL[Hsite,site] += 1
    THL[site,Hsite] += 1
THL = mbt.row_normalize_matrix(THL)

In [ ]:
%%time
HL = pr_peaks.JumpingModel(THL,site_taus)
for mu in mus :
    mbt.log_message('HL','mu = %d'%(mu))
    omega_t_initial = pr_peaks.init_omega_t(N,n,mu)
    HL.run(nsteps,mu,sigma,omega_t_initial)

In [ ]:
H_to_L(HL,[Hsite],Lsites)
H_to_L(flat,[Hsite],Lsites)

In [ ]:
plt.plot(mus,flat.H_to_L,'r')
plt.plot(mus,HL.H_to_L,'b')
plt.xlabel(r'$\mu$')
plt.ylabel('H to L ratio')
plt.axhline(y=1.0,linestyle='--',color='k')
plt.show()

This doesn't work.

Another possibility: there are many H sites, and the number of searchers is smaller than the number of available sites.

## Non-saturating H sites

Let's study this idea.

In [ ]:
# prepare site_taus
n = 5000
nH = 500
np.random.seed(6589)
# Hsites and Lsites
Hsites = np.random.choice(n,size=nH,replace=False)
Lsites = range(n)
for site in Hsites :
    Lsites.remove(site)
site_taus = 2.0*np.ones(n)
site_taus[Hsites] = 20.0
# plot
fig = plt.figure(figsize=(10,3))
ax = plt.subplot(111)
mbt.line_plot(ax,np.arange(n),site_taus)
plt.show()

In [ ]:
# do the simulations
N = 100
nsteps = 1000
Tflat = np.ones((n,n))/n
flat = pr_peaks.JumpingModel(Tflat,site_taus)
mus = np.array([1,10,25,50,100])
for mu in mus :
    mbt.log_message('Flat','mu = %d'%(mu))
    omega_t_initial = pr_peaks.init_omega_t(N,n,mu,sigma=5)
    flat.run(nsteps,mu,sigma,omega_t_initial)
H_to_L(flat,Hsites,Lsites)

In [ ]:
THL = np.ones((n,n))
for Hsite in Hsites :
    for Lsite in Lsites :
        THL[Hsite,Lsite] += 1
        THL[Lsite,Hsite] += 1
THL = mbt.row_normalize_matrix(THL)

In [ ]:
%%time
HL = pr_peaks.JumpingModel(THL,site_taus)
for mu in mus :
    mbt.log_message('HL','mu = %d'%(mu))
    omega_t_initial = pr_peaks.init_omega_t(N,n,mu)
    HL.run(nsteps,mu,sigma,omega_t_initial)
H_to_L(HL,Hsites,Lsites)

In [ ]:
plt.plot(mus,flat.H_to_L,'r')
plt.plot(mus,HL.H_to_L,'b')
plt.xlabel(r'$\mu$')
plt.ylabel('H to L ratio')
plt.axhline(y=1.0,linestyle='--',color='k')
plt.show()